In [1]:
import math
import numpy as np
import torch
import pkbar
from unet3d.config import *
from utils.Visualization import ImageSliceViewer3D
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import glob
from torch.optim import Adam
from unet3d.dataset import SAIADDataset
from torch.utils.data import DataLoader
from patchify import patchify
import nrrd

%load_ext autoreload
%autoreload 2
%matplotlib inline
excl_patients = ['SAIAD 1','SAIAD 2','SAIAD 5','SAIAD 7','SAIAD 9','SAIAD 11','SAIAD 12','SAIAD 13','SAIAD 14',
                    'SAIAD 15','SAIAD 20 vasculaire', 'SAIAD 20 BIS vasculaire', 'SAIAD 19 BIS vasculaire']


In [2]:
dataset = SAIADDataset(excl_patients=excl_patients, batch_size=8, epochs=100, n_batches=100, load_data_to_memory=True)

Fetching patients probabilities...


100%|██████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  3.24it/s]


In [5]:
# DataLoader test
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, pin_memory=True, num_workers=2)

/Work/Users/acharneca/.conda/envs/saiad-pytorch/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
i=0
for imgs, labels in tqdm(dataloader):
    i+=1
    if i==10:
        break


  0%|                                                                        | 0/100 [00:00<?, ?it/s]/Work/Users/acharneca/.conda/envs/saiad-pytorch/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  9%|█████▊                                                          | 9/100 [00:07<01:10,  1.28it/s]


In [21]:
n=0
ImageSliceViewer3D(np.array(imgs[n,0]), np.array(torch.argmax(labels[n], dim=0)))

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [20]:
labels[n].shape

torch.Size([5, 96, 96, 96])

In [26]:
torch.mean(torch.argmax(labels[n], dim=0).float())

tensor(0.5464)

In [ ]:
e